# Countdown Numbers Game
https://en.wikipedia.org/wiki/Countdown_(game_show)#Numbers_round
***

The Countdown game is a british game show involving word and number tasks. Two contestants compete against each other in a series of rounds. One of these rounds is the numbers game.

There are two groups of numbers, one being large numbers containing <b> 25, 50, 75 and 100 </b> and small numbers containing a selection of <b> 1 to 10 twice </b>. The contestants need to decide six numbers from the large and small numbers in any combination they'd like. This means they could choose up to four or none large numbers if they desire and make up the rest with small numbers. 

A target number is randomly generated which is a three digit number which can be between 101-999 inclusive. They have to use arithmetic using the chosen numbers to get to the target number. 

## Rules
***
* Only basic arithmetic can be used which involes
    * Addition
    * Subtraction
    * Multiplication
    * Division
* Fractions of numbers are not allowed
* Numbers cannot become negative
* Numbers cannot be used twice
* Concatenating numbers is not allowed
* The solution does not require that you use all the numbers

<br>

## Complexity of the Numbers Game
***
The numbers game is incredibly complex due to the amount of possible combinations of solutions.

In [4]:
# Lets assume we have just six random numbers
L1 = [1, 2, 3, 4, 5, 6]
# This also assumes that we'd have five different operators associated with these numbers
L2 = ['-', '+', '*', '/', '+']

In [5]:
# There are many ways a simple sum can be interpreted
# Take for example this sum 6 - 4 - 2
# This can be interpreted differently
sum1 = (6 - 1) - 2
sum2 = 6 - (1 - 2)
sum1, sum2

(3, 7)

So we have to come up with a way to represent these sums in a way that they are interpeted the correct way

### Reverse Polish Notation RPN
***

This saves the interperter or compiler an awful lot of work when it comes to figuring out how to work with these sums

In [6]:
# For example if we take the previous two sums
sum1 = (6 - 1) - 2
# would be written as 61-2-
sum2 = 6 - (1 - 2)
# would be wrriten as 612--

So the order of where the symbols come up denotes how the sum would be handled 

Which is one problem solved

### Possible combinations
***

In [7]:
# If we take the two lists we had from earlier on
L1, L2

([1, 2, 3, 4, 5, 6], ['-', '+', '*', '/', '+'])

In [8]:
# There are eleven elements between the two sets
# if we were to shuffle all these elements we'd have
import math
math.factorial(11)

39916800

We'd have 39,916,800 possible combinations, and that's not including all the combinations of operators involved.

In [9]:
# If we get all the possible sets of operators, we'd have 54 differnet sets
# Which means we'd have 
39916800 * 54

2155507200

We'd have 2.155 billion possible combinations, and that's not even including all the combinations of numbers involved. Where the number of combinations becomes astronomically large

<br>

## Python Solution to solve the Countdown Numbers Game
***

In [10]:
# Random Number Generator
import random
# To deal with permutations and combinations
import itertools as it
from more_itertools import distinct_permutations
# Operators as functions
import operator

<br>

### Acquiring the numbers randomly for our countdown game
***

In [11]:
# Large numbers
large = [25, 50, 75, 100]
large

[25, 50, 75, 100]

In [12]:
# Small numbers
small = sorted(list(range(1, 11)) * 2)
small

[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10]

In [13]:
# The number of large numbers to pick 
no_large = random.randrange(0, 5)
no_large

3

In [14]:
# Selecting no_large large numbers at random
large_rand = random.sample(large, no_large)
large_rand

[100, 50, 75]

In [15]:
# Selecting (6 - no_large) small numbers at random
small_rand = random.sample(small, 6 - no_large)
small_rand

[5, 10, 3]

In [16]:
# Putting all random numbers in a list
countdown_numbers = large_rand + small_rand
countdown_numbers 

[100, 50, 75, 5, 10, 3]

In [17]:
# Picking a random target number
target = random.randrange(101, 1000)
target

778

In [18]:
# Making an all in one function to generate numbers for our countdown game
def new_numbers_game():
    """ Returns six numbers and a target number representing a Countdown numbers game. """
    no_large = random.randrange(0, 5)
    large_rand = random.sample([25, 50, 75, 100], no_large)
    small_rand = random.sample(list(range(1, 11)) * 2, (6 - no_large))
    countdown_numbers = large_rand + small_rand
    target = random.randrange(101, 1000)
    return countdown_numbers, target

In [19]:
# Random numbers game
countdown_numbers, target = new_numbers_game()
countdown_numbers, target

([25, 100, 50, 6, 8, 3], 797)

In [20]:
# Evaluating RPN expression
def evaluate_rpn(rpn):
    stack = []
    # Loop through rpn an item at a time
    for i in rpn:
        # check if its a number
        if isinstance(i, int):
            # Append to the stack.
            stack = stack + [i]
        else:
            # Pop from stack twice
            right = stack[-1]
            stack = stack[:-1]
            left = stack[-1]
            stack = stack[:-1]
            # Push operator applied to stack elements
            stack = stack + [i(left, right)]
    # should only be one item on the stack
    return stack[0]

In [21]:
def patterns(numbers, operators):
  # Check if there is no way to partition further.
  if len(numbers) == 1:
    yield numbers
  # Loop through all the ways to partition L into two non-empty sublists.
  for i in range(1, len(numbers)):
    # Slice the list using i.
    for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
      # Yield the next operator applied to the sublists.
      yield [*left, *right, operators[0]]

In [29]:
# Search for the target number solution using RPN notation and permutations of numbers and operators.
def search_rpn(numbers, operators, target):
	print("The target is {}".format(target))

	for i in patterns(numbers, operators):
		print(i)
		result = evaluate_rpn(i);
		print("The Result is {}".format(result))

		if result == target:
			return i

solution = search_rpn(countdown_numbers, [operator.add, operator.mul, operator.truediv, operator.sub, operator.add], target)

The target is 797
[25, 100, 50, 6, 8, 3, <built-in function add>, <built-in function sub>, <built-in function truediv>, <built-in function mul>, <built-in function add>]
The Result is -975.0
[25, 100, 50, 6, 8, <built-in function add>, 3, <built-in function sub>, <built-in function truediv>, <built-in function mul>, <built-in function add>]
The Result is 479.54545454545456
[25, 100, 50, 6, <built-in function sub>, 8, 3, <built-in function add>, <built-in function truediv>, <built-in function mul>, <built-in function add>]
The Result is 425.0
[25, 100, 50, 6, 8, <built-in function add>, <built-in function sub>, 3, <built-in function truediv>, <built-in function mul>, <built-in function add>]
The Result is 1225.0
[25, 100, 50, 6, <built-in function add>, 8, <built-in function sub>, 3, <built-in function truediv>, <built-in function mul>, <built-in function add>]
The Result is 1625.0
[25, 100, 50, <built-in function truediv>, 6, 8, 3, <built-in function add>, <built-in function sub>, <bui

<br>

## Resources
***

1. [Countdown Wikipedia](https://en.wikipedia.org/wiki/Countdown_(game_show))
2. [Brute Forcing The Countdown Numbers Game - Computerphile](https://www.youtube.com/watch?v=cVMhkqPP2YI&t=435s&ab_channel=Computerphile)
3. [Countdown Game Show DataGenetics](https://datagenetics.com/blog/august32014/index.html)
4. [Functional Programming in Python: When and How to Use It](https://realpython.com/python-functional-programming/)
5. [Countdown numbers game (Solution generator)](https://codereview.stackexchange.com/questions/190533/countdown-numbers-game-solution-generator)